In [21]:
Types of Recommendation System:
    1. Content Based - on based of content similarity, it will recommend content
    2. Collaborative Filtering based - on users interest basis, it will recommend content. 
    User a & b are very similar, both watched same movies they given similar ratings.
    Say user a watched m2, if a loves m2 then recommend m2 to b also
    EG: Facebook
    3. Hybrid
    
Project Flow:
    Data -> Pre-processing -> Model -> Website convert -> Deploy

SyntaxError: invalid syntax (<ipython-input-21-e5b99a75f08a>, line 1)

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

from sklearn.feature_extraction.text import CountVectorizer
#tfidf
from sklearn.feature_extraction.text import TfidfVectorizer
# gensim
from gensim import models

import nltk
from nltk.stem.porter import PorterStemmer

from sklearn.metrics.pairwise import cosine_similarity

In [2]:
#tmdb the movie data base
movies = pd.read_csv("D:\\Project\\Movie Recommendation System\\tmdb_movies_data.csv")

In [3]:
movies.head()

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,keywords,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
0,135397,tt0369610,32.985763,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,http://www.jurassicworld.com/,Colin Trevorrow,The park is open.,monster|dna|tyrannosaurus rex|velociraptor|island,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/2015,5562,6.5,2015,137999939.3,1.392446e+09
1,76341,tt1392190,28.419936,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,http://www.madmaxmovie.com/,George Miller,What a Lovely Day.,future|chase|post-apocalyptic|dystopia|australia,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/2015,6185,7.1,2015,137999939.3,3.481613e+08
2,262500,tt2908446,13.112507,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,http://www.thedivergentseries.movie/#insurgent,Robert Schwentke,One Choice Can Destroy You,based on novel|revolution|dystopia|sequel|dyst...,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,3/18/2015,2480,6.3,2015,101199955.5,2.716190e+08
3,140607,tt2488496,11.173104,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,http://www.starwars.com/films/star-wars-episod...,J.J. Abrams,Every generation has a story.,android|spaceship|jedi|space opera|3d,Thirty years after defeating the Galactic Empi...,136,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,12/15/2015,5292,7.5,2015,183999919.0,1.902723e+09
4,168259,tt2820852,9.335014,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,http://www.furious7.com/,James Wan,Vengeance Hits Home,car race|speed|revenge|suspense|car,Deckard Shaw seeks revenge against Dominic Tor...,137,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,4/1/2015,2947,7.3,2015,174799923.1,1.385749e+09


In [4]:
print(movies.shape)

(10866, 21)


In [5]:
print(movies.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10866 entries, 0 to 10865
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    10866 non-null  int64  
 1   imdb_id               10856 non-null  object 
 2   popularity            10866 non-null  float64
 3   budget                10866 non-null  int64  
 4   revenue               10866 non-null  int64  
 5   original_title        10866 non-null  object 
 6   cast                  10790 non-null  object 
 7   homepage              2936 non-null   object 
 8   director              10822 non-null  object 
 9   tagline               8042 non-null   object 
 10  keywords              9373 non-null   object 
 11  overview              10862 non-null  object 
 12  runtime               10866 non-null  int64  
 13  genres                10843 non-null  object 
 14  production_companies  9836 non-null   object 
 15  release_date       

In [6]:
# columns to keep, aim - create content based recommendation system, which means we need to get tags
# budget - means if we like a movie with high-budget, it doesn't means the next movie with high budget we will like\
# genres(IMP) - people like genres like sci-fi, horror, thriller
# homepage - not important
# id(IMP) - needed for webapp
# keywords(IMP) - tags
# original_title(IMP) - original title
# overview(IMP) - if 2 movies summary is similar, then these 2 movies also similar
# popularity - drop as of now
# production companies - when we recommend movies, we don't recommend on production basis, we recommend on actors, director
# release-year - can be important
# tagline - very vague, movies can be different, tagline can be totally diff - overview is solving this purpose
# vote count & avg - we are skipping numeric col
# cast(IMP) we will keep - if we like SRK, we will watch more movies of SRK
# director(IMP) - if we like one director Rohit Shetty, we tend to like same movies of him

In [7]:
# So we are keeping
movies[['id', 'original_title', 'overview', 'genres', 'keywords', 'cast', 'director']].head()

,id,original_title,overview,genres,keywords,cast,director
0,135397,Jurassic World,Twenty-two years after the events of Jurassic ...,Action|Adventure|Science Fiction|Thriller,monster|dna|tyrannosaurus rex|velociraptor|island,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow
1,76341,Mad Max: Fury Road,An apocalyptic story set in the furthest reach...,Action|Adventure|Science Fiction|Thriller,future|chase|post-apocalyptic|dystopia|australia,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller
2,262500,Insurgent,Beatrice Prior must confront her inner demons ...,Adventure|Science Fiction|Thriller,based on novel|revolution|dystopia|sequel|dyst...,Shailene Woodley|Theo James|Kate Winslet|Ansel...,Robert Schwentke
3,140607,Star Wars: The Force Awakens,Thirty years after defeating the Galactic Empi...,Action|Adventure|Science Fiction|Fantasy,android|spaceship|jedi|space opera|3d,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,J.J. Abrams
4,168259,Furious 7,Deckard Shaw seeks revenge against Dominic Tor...,Action|Crime|Thriller,car race|speed|revenge|suspense|car,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,James Wan


In [8]:
movies = movies[['id', 'original_title', 'overview', 'genres', 'keywords', 'cast', 'director']]
movies.head()

,id,original_title,overview,genres,keywords,cast,director
0,135397,Jurassic World,Twenty-two years after the events of Jurassic ...,Action|Adventure|Science Fiction|Thriller,monster|dna|tyrannosaurus rex|velociraptor|island,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow
1,76341,Mad Max: Fury Road,An apocalyptic story set in the furthest reach...,Action|Adventure|Science Fiction|Thriller,future|chase|post-apocalyptic|dystopia|australia,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller
2,262500,Insurgent,Beatrice Prior must confront her inner demons ...,Adventure|Science Fiction|Thriller,based on novel|revolution|dystopia|sequel|dyst...,Shailene Woodley|Theo James|Kate Winslet|Ansel...,Robert Schwentke
3,140607,Star Wars: The Force Awakens,Thirty years after defeating the Galactic Empi...,Action|Adventure|Science Fiction|Fantasy,android|spaceship|jedi|space opera|3d,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,J.J. Abrams
4,168259,Furious 7,Deckard Shaw seeks revenge against Dominic Tor...,Action|Crime|Thriller,car race|speed|revenge|suspense|car,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,James Wan


In [9]:
#how our df will look in future
#movie id, movie_title, tags(for tags we are going to merge the other columns)
#top 3 from cast

In [10]:
#check for missing values
movies.isna().sum()

id                   0
original_title       0
overview             4
genres              23
keywords          1493
cast                76
director            44
dtype: int64

In [11]:
movies[movies.keywords.isna()].head()

,id,original_title,overview,genres,keywords,cast,director
130,245706,True Story,A drama centered around the relationship betwe...,Crime|Drama|Mystery,NaN,Jonah Hill|James Franco|Felicity Jones|Maria D...,Rupert Goold
149,346808,Momentum,"When Alex, an infiltration expert with a secre...",Thriller|Action,NaN,Olga Kurylenko|Morgan Freeman|James Purefoy|Je...,Stephen S. Campanelli
154,314405,Il racconto dei racconti,"A fantasy film with horror elements, ""The Tale...",Romance|Fantasy|Horror,NaN,Salma Hayek|Vincent Cassel|John C. Reilly|Toby...,Matteo Garrone
155,282984,Irrational Man,"On a small town college campus, a philosophy p...",Mystery|Drama,NaN,Emma Stone|Joaquin Phoenix|Jamie Blackley|Park...,Woody Allen
160,304372,The Perfect Guy,Leah Vaughn appears to have the ideal life. Sh...,Drama|Thriller,NaN,Michael Ealy|Sanaa Lathan|Morris Chestnut|Kath...,David M. Rosenthal


In [12]:
movies.dropna(inplace=True)
movies.shape

(9306, 7)

In [13]:
#check for duplicates
movies.duplicated().sum()

1

In [14]:
movies.drop_duplicates(inplace=True)

In [15]:
movies.reset_index(drop=True, inplace=True)

In [16]:
#Pre-processing

In [17]:
#converting into a list of genres
def pre_process_genre(data):
    splitted_data = data['genres'].split('|')
    return splitted_data

In [18]:
movies['genres'] = movies.apply(lambda row: pre_process_genre(row), axis=1)
movies['genres'].head()

0    [Action, Adventure, Science Fiction, Thriller]
1    [Action, Adventure, Science Fiction, Thriller]
2            [Adventure, Science Fiction, Thriller]
3     [Action, Adventure, Science Fiction, Fantasy]
4                         [Action, Crime, Thriller]
Name: genres, dtype: object

In [19]:
#converting into a list of keywords
def pre_process_keywords(data):
    splitted_data = data['keywords'].split('|')
    return splitted_data

In [20]:
movies['keywords'] = movies.apply(lambda row: pre_process_keywords(row), axis=1)
movies['keywords'].head()

0    [monster, dna, tyrannosaurus rex, velociraptor...
1    [future, chase, post-apocalyptic, dystopia, au...
2    [based on novel, revolution, dystopia, sequel,...
3          [android, spaceship, jedi, space opera, 3d]
4            [car race, speed, revenge, suspense, car]
Name: keywords, dtype: object

In [21]:
#pre-processing & extracting top 3 casts
def pre_process_cast(data):
    splitted_data = data['cast'].split('|')
    if len(splitted_data) < 3:
        return splitted_data
    return splitted_data[:3]

In [22]:
movies['cast'] = movies.apply(lambda row: pre_process_cast(row), axis=1)
movies['cast'].head()

0    [Chris Pratt, Bryce Dallas Howard, Irrfan Khan]
1     [Tom Hardy, Charlize Theron, Hugh Keays-Byrne]
2       [Shailene Woodley, Theo James, Kate Winslet]
3        [Harrison Ford, Mark Hamill, Carrie Fisher]
4           [Vin Diesel, Paul Walker, Jason Statham]
Name: cast, dtype: object

In [23]:
#we are going to convert movies overview to list
movies['overview'][0]

'Twenty-two years after the events of Jurassic Park, Isla Nublar now features a fully functioning dinosaur theme park, Jurassic World, as originally envisioned by John Hammond.'

In [24]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [25]:
movies.head()

,id,original_title,overview,genres,keywords,cast,director
0,135397,Jurassic World,"[Twenty-two, years, after, the, events, of, Ju...","[Action, Adventure, Science Fiction, Thriller]","[monster, dna, tyrannosaurus rex, velociraptor...","[Chris Pratt, Bryce Dallas Howard, Irrfan Khan]",Colin Trevorrow
1,76341,Mad Max: Fury Road,"[An, apocalyptic, story, set, in, the, furthes...","[Action, Adventure, Science Fiction, Thriller]","[future, chase, post-apocalyptic, dystopia, au...","[Tom Hardy, Charlize Theron, Hugh Keays-Byrne]",George Miller
2,262500,Insurgent,"[Beatrice, Prior, must, confront, her, inner, ...","[Adventure, Science Fiction, Thriller]","[based on novel, revolution, dystopia, sequel,...","[Shailene Woodley, Theo James, Kate Winslet]",Robert Schwentke
3,140607,Star Wars: The Force Awakens,"[Thirty, years, after, defeating, the, Galacti...","[Action, Adventure, Science Fiction, Fantasy]","[android, spaceship, jedi, space opera, 3d]","[Harrison Ford, Mark Hamill, Carrie Fisher]",J.J. Abrams
4,168259,Furious 7,"[Deckard, Shaw, seeks, revenge, against, Domin...","[Action, Crime, Thriller]","[car race, speed, revenge, suspense, car]","[Vin Diesel, Paul Walker, Jason Statham]",James Wan


In [26]:
# we want to convert all these lists into one list and then convert that list to one paragraph

In [27]:
# now we want to remove any space between the words(transformation)
# Eg - Chris Pratt -> ChrisPratt, if not Chris and Pratt becomes different entities, hence both becomes different tags
# For eg, we have another person called Chris Evans, hence when we want to see Chris Prats, 
# model will become confuse which Chriss to recommend

In [28]:
#transformation
movies['genres'] = movies.genres.apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies.keywords.apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies.cast.apply(lambda x:[i.replace(" ","") for i in x])
movies['director'] = movies.director.apply((lambda x: x.replace(" ", "")))

In [29]:
def add_list_bracket(data):
    splitted_data = data['director']
    return [splitted_data]
movies['director'] = movies.apply(lambda row: add_list_bracket(row), axis=1)

In [30]:
movies.head()

,id,original_title,overview,genres,keywords,cast,director
0,135397,Jurassic World,"[Twenty-two, years, after, the, events, of, Ju...","[Action, Adventure, ScienceFiction, Thriller]","[monster, dna, tyrannosaurusrex, velociraptor,...","[ChrisPratt, BryceDallasHoward, IrrfanKhan]",[ColinTrevorrow]
1,76341,Mad Max: Fury Road,"[An, apocalyptic, story, set, in, the, furthes...","[Action, Adventure, ScienceFiction, Thriller]","[future, chase, post-apocalyptic, dystopia, au...","[TomHardy, CharlizeTheron, HughKeays-Byrne]",[GeorgeMiller]
2,262500,Insurgent,"[Beatrice, Prior, must, confront, her, inner, ...","[Adventure, ScienceFiction, Thriller]","[basedonnovel, revolution, dystopia, sequel, d...","[ShaileneWoodley, TheoJames, KateWinslet]",[RobertSchwentke]
3,140607,Star Wars: The Force Awakens,"[Thirty, years, after, defeating, the, Galacti...","[Action, Adventure, ScienceFiction, Fantasy]","[android, spaceship, jedi, spaceopera, 3d]","[HarrisonFord, MarkHamill, CarrieFisher]",[J.J.Abrams]
4,168259,Furious 7,"[Deckard, Shaw, seeks, revenge, against, Domin...","[Action, Crime, Thriller]","[carrace, speed, revenge, suspense, car]","[VinDiesel, PaulWalker, JasonStatham]",[JamesWan]


In [31]:
#tags concat of overview, genres, keywords, cast & director
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['director']

In [32]:
new_df = movies[['id', 'original_title', 'tags']]
new_df.head()

,id,original_title,tags
0,135397,Jurassic World,"[Twenty-two, years, after, the, events, of, Ju..."
1,76341,Mad Max: Fury Road,"[An, apocalyptic, story, set, in, the, furthes..."
2,262500,Insurgent,"[Beatrice, Prior, must, confront, her, inner, ..."
3,140607,Star Wars: The Force Awakens,"[Thirty, years, after, defeating, the, Galacti..."
4,168259,Furious 7,"[Deckard, Shaw, seeks, revenge, against, Domin..."


In [33]:
#converting the tags list to strings to make it into paragraph
new_df['tags'] = new_df['tags'].apply(lambda x : " ".join(x))

<ipython-input-33-03de5f7ac922>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x : " ".join(x))


In [34]:
new_df.head()

,id,original_title,tags
0,135397,Jurassic World,Twenty-two years after the events of Jurassic ...
1,76341,Mad Max: Fury Road,An apocalyptic story set in the furthest reach...
2,262500,Insurgent,Beatrice Prior must confront her inner demons ...
3,140607,Star Wars: The Force Awakens,Thirty years after defeating the Galactic Empi...
4,168259,Furious 7,Deckard Shaw seeks revenge against Dominic Tor...


In [35]:
new_df.tags[1]

"An apocalyptic story set in the furthest reaches of our planet, in a stark desert landscape where humanity is broken, and most everyone is crazed fighting for the necessities of life. Within this world exist two rebels on the run who just might be able to restore order. There's Max, a man of action and a man of few words, who seeks peace of mind following the loss of his wife and child in the aftermath of the chaos. And Furiosa, a woman of action and a woman who believes her path to survival may be achieved if she can make it across the desert back to her childhood homeland. Action Adventure ScienceFiction Thriller future chase post-apocalyptic dystopia australia TomHardy CharlizeTheron HughKeays-Byrne GeorgeMiller"

In [36]:
#converting all tags to lower case
new_df['tags'] = new_df['tags'].apply(lambda x : x.lower())

<ipython-input-36-4467a02ead9f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x : x.lower())


In [37]:
new_df.head()

,id,original_title,tags
0,135397,Jurassic World,twenty-two years after the events of jurassic ...
1,76341,Mad Max: Fury Road,an apocalyptic story set in the furthest reach...
2,262500,Insurgent,beatrice prior must confront her inner demons ...
3,140607,Star Wars: The Force Awakens,thirty years after defeating the galactic empi...
4,168259,Furious 7,deckard shaw seeks revenge against dominic tor...


In [38]:
# action -> actions, actor -> actors, these should be same word not different, we will apply stemings
# Eg: ['love', 'loving', 'loved'] ----> after steming -----> ['love', 'love', 'love']
# 1930s, 1942 etc have meaning means it has story related to these dates

In [39]:
ps = PorterStemmer()

In [40]:
def stem(row):
    y = []
    text = row['tags']
    for i in text.split():
        y.append(ps.stem(i))
    
    return " ".join(y)

In [41]:
new_df['tags'] = new_df.apply(lambda row: stem(row), axis=1)

<ipython-input-41-b84fba92e472>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df.apply(lambda row: stem(row), axis=1)


In [42]:
# Out of 10k movies how do we know which top 5 are similar,
# we need to find similarity between two tags
# we need vectorization
# tags -> vectors 
# vector is nothing but a coordinate having x & y values
# when we convert tags to vector, every movie will become a vector
# we will have 10000 vectors, now which movie will we recommend, we will recommend closest vector
# text -> vectors:
# text vectorization process is bag of words, tf-idf, word-2-vec
# Bag of words:
# 1. concat all tags together (Eg: large_tag = tag1 + tag2 + tag3..)
# 2. now, from Large_tag, we need 10k most common words(calculate frequency) say w1, w2, w3
# 3. now, again we will go to our tag and check how many times a word repeats from w1 to wn
# Eg - if w1 is action, then how many times action repeats in tag1, w2 is future ..
# Movie | w1 | w2 | w3  | w10k, w1 can be action , w2 can be Salman Khan etc..
#  m1   | 5  | 2  |  1  |  10
#  m2   | 4  |  5 |  1  |  100
# m10k  | 5  | 1  |  3  | 2
# shape (10k,10k), hence 10k dimensions, each movie M(i) is vector of 10k dimension
# while doing vectorization, we won't consider stop-words(common words -> is, and, from, was, are, of)

In [43]:
#cv = CountVectorizer(max_features = 5000, stop_words='english')#max f- 5000

In [44]:
tfidf = TfidfVectorizer(max_features = 5000, stop_words='english')

In [45]:
#cv.fit_transform(new_df['tags']).toarray().shape#convert sparse matrix to numpy array

In [46]:
tfidf.fit_transform(new_df['tags']).toarray().shape#convert sparse matrix to numpy array

(9305, 5000)

In [47]:
vectors = tfidf.fit_transform(new_df['tags']).toarray() #cv.fit_transform(new_df['tags']).toarray()
vectors

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [48]:
vectors[0]

array([0., 0., 0., ..., 0., 0., 0.])

In [49]:
tfidf.get_feature_names()#cv.get_feature_names() #these are all the 5k words

['000',
 '10',
 '100',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '16th',
 '17',
 '18',
 '18th',
 '19',
 '1920',
 '1930',
 '1940',
 '1950',
 '1950s',
 '1960',
 '1960s',
 '1969',
 '1970',
 '1970s',
 '1972',
 '1974',
 '1976',
 '1980',
 '1980s',
 '1984',
 '1986',
 '1987',
 '1990',
 '1994',
 '1999',
 '19th',
 '20',
 '200',
 '2000',
 '2001',
 '2003',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '20th',
 '21',
 '21st',
 '23',
 '24',
 '25',
 '30',
 '300',
 '35',
 '3d',
 '40',
 '50',
 '500',
 '60',
 '70',
 '80',
 '90',
 'aaron',
 'aaroneckhart',
 'abandon',
 'abbi',
 'abduct',
 'abigailbreslin',
 'abil',
 'abl',
 'aboard',
 'abov',
 'absolut',
 'absorb',
 'abus',
 'abuse',
 'academ',
 'academi',
 'academy',
 'accept',
 'access',
 'accid',
 'accident',
 'acclaim',
 'accompani',
 'accomplish',
 'accord',
 'account',
 'accus',
 'ace',
 'achiev',
 'acquaint',
 'acquir',
 'act',
 'action',
 'activ',
 'activist',
 'actor',
 'actress',
 'actual',
 'ad',
 'adam',
 'adamsandl',
 'adapt',
 'add',
 'add

In [50]:
# we have 9305 movies vectors
# now we need to calculate the difference between the movies each other, more distance - less similarity and vice versa
# no euclidean distance, we will calculate cosine(angle between them, if angle small then small distance)
# whenever high dimensionality data, euclidean distance is not a good measure, cosine distance (theta between two vectors)
# distance inversly proportion to similarity

In [51]:
#cosine similarity (0-1,1-means more similairty, 0 means low )
cosine_similarity(vectors).shape #each movie with every movie, like correlation matrix

(9305, 9305)

In [52]:
similarity = cosine_similarity(vectors)
similarity

array([[1.        , 0.04932207, 0.0392262 , ..., 0.        , 0.01730779,
        0.        ],
       [0.04932207, 1.        , 0.0637909 , ..., 0.01098352, 0.03997705,
        0.00279582],
       [0.0392262 , 0.0637909 , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.01098352, 0.        , ..., 1.        , 0.00502005,
        0.00411825],
       [0.01730779, 0.03997705, 0.        , ..., 0.00502005, 1.        ,
        0.        ],
       [0.        , 0.00279582, 0.        , ..., 0.00411825, 0.        ,
        1.        ]])

In [134]:
similarity[0]

array([1.        , 0.04932207, 0.0392262 , ..., 0.        , 0.01730779,
       0.        ])

In [135]:
sorted(list(enumerate(similarity[0])), reverse=True, key = lambda x:x[1])[1:6]#movie|, distance

[(8697, 0.5511502472536572),
 (4396, 0.32973938557419474),
 (8668, 0.3133231642536092),
 (37, 0.30380564713027813),
 (5702, 0.266098830759798)]

In [136]:
def recommend(movie):
    movie_index = new_df[new_df['original_title'] == movie].index[0]
    distances = similarity[movie_index]
    #sorting array
    movies_list = sorted(list(enumerate(distances)), reverse=True, key = lambda x:x[1])[1:6]
    
    for movies in movies_list:
        print(new_df.iloc[movies[0]].original_title)

In [137]:
recommend("Batman")

Batman Beyond: Return of the Joker
Batman Returns
Batman & Robin
Batman Unlimited: Monster Mayhem
The Dark Knight Rises


tfidf works better than count vectorizer

In [138]:
import pickle

In [139]:
pickle.dump(new_df.to_dict(), open('D:\\Project\\Movie Recommendation System\\movies.pkl', 'wb'))

In [140]:
pickle.dump(similarity, open('D:\\Project\\Movie Recommendation System\\similarity.pkl', 'wb'))

#Compressing pickle size 

In [141]:
import bz2
import pickle
import _pickle as cPickle

In [143]:
def compressed_pickle(title, data):
    with bz2.BZ2File(title + '.pbz2' , 'w') as f:
        cPickle.dump(data,f)

In [144]:
compressed_pickle('D:\\Project\\Movie Recommendation System\\similarity_compressed', similarity) 

In [3]:
#reading from azure

In [19]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, __version__
try:
    print("Azure Blob Storage v" + __version__ + " - Python quickstart sample")

    # Quick start code goes here

except Exception as ex:
    print('Exception:')
    print(ex)

Azure Blob Storage v12.9.0 - Python quickstart sample


In [ ]:
STORAGEACCOUNTURL= ""
STORAGEACCOUNTKEY= <storage_account_key>
LOCALFILENAME= <local_file_name>
CONTAINERNAME= <container_name>
BLOBNAME= <blob_name>
connection string =- 'DefaultEndpointsProtocol=https;AccountName=arjunprasadsarkhel;AccountKey=V0FuvezEvjBVwZo7X0o2hq+v9lgNasyKQH/VKOoQRqaoURORgMPWhM9HbQj+7Sx5J7wvkLBOtd5k+ASttj4qkg==;EndpointSuffix=core.windows.net'

In [22]:
import pickle
def get_weights_blob(blob_name):
    connection_string = 'DefaultEndpointsProtocol=https;AccountName=arjunprasadsarkhel;AccountKey=V0FuvezEvjBVwZo7X0o2hq+v9lgNasyKQH/VKOoQRqaoURORgMPWhM9HbQj+7Sx5J7wvkLBOtd5k+ASttj4qkg==;EndpointSuffix=core.windows.net'
    container_name = 'movierecommendation'
    blob_client = BlobClient.from_connection_string(connection_string, container_name, blob_name)
    downloader = blob_client.download_blob(0)

    # Load to pickle
    b = downloader.readall()
    weights = pickle.loads(b)

    return weights

In [10]:
connection_string = 'DefaultEndpointsProtocol=https;AccountName=arjunprasadsarkhel;AccountKey=V0FuvezEvjBVwZo7X0o2hq+v9lgNasyKQH/VKOoQRqaoURORgMPWhM9HbQj+7Sx5J7wvkLBOtd5k+ASttj4qkg==;EndpointSuffix=core.windows.net'
container_name = 'movierecommendation'
blob_name = 'similarity.pkl'
blob_client = BlobClient.from_connection_string(connection_string, container_name, blob_name)

In [23]:
%%time
get_weights_blob(blob_name = 'similarity.pkl')

Wall time: 8min 21s


array([[1.        , 0.04932207, 0.0392262 , ..., 0.        , 0.01730779,
        0.        ],
       [0.04932207, 1.        , 0.0637909 , ..., 0.01098352, 0.03997705,
        0.00279582],
       [0.0392262 , 0.0637909 , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.01098352, 0.        , ..., 1.        , 0.00502005,
        0.00411825],
       [0.01730779, 0.03997705, 0.        , ..., 0.00502005, 1.        ,
        0.        ],
       [0.        , 0.00279582, 0.        , ..., 0.00411825, 0.        ,
        1.        ]])

In [54]:
similarity

array([[1.        , 0.04932207, 0.0392262 , ..., 0.        , 0.01730779,
        0.        ],
       [0.04932207, 1.        , 0.0637909 , ..., 0.01098352, 0.03997705,
        0.00279582],
       [0.0392262 , 0.0637909 , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.01098352, 0.        , ..., 1.        , 0.00502005,
        0.00411825],
       [0.01730779, 0.03997705, 0.        , ..., 0.00502005, 1.        ,
        0.        ],
       [0.        , 0.00279582, 0.        , ..., 0.00411825, 0.        ,
        1.        ]])

In [55]:
similarity.shape

(9305, 9305)

In [58]:
import tables
h5_file = tables.open_file('D:\\Project\\Movie Recommendation System\\my_array.h5', mode='w', titel='many large arrays')

for n, arr in enumerate(similarity):
    h5_file.create_array('/', 'my_array{}'.format(n), arr)
h5_file.close()

In [61]:
from joblib import dump
filename = 'D:\\Project\\Movie Recommendation System\\ss'
dump(similarity, filename, compress=)

['D:\\Project\\Movie Recommendation System\\ss']

In [ ]:
#test

In [79]:
from functools import wraps

def callonce(f):

    @wraps(f)
    def wrapper(*args, **kwargs):
        if not wrapper.called:
            wrapper.called = True
            return f(*args, **kwargs)
        print ('Function already called once.')
    wrapper.called = False
    return wrapper

In [81]:
@callonce
def func():
    print ("Creating Table")

In [84]:
func()
func()
func()

Function already called once.
Function already called once.
Function already called once.


#count vect
#tdfidf
#google news